# Sentinel-2 analysis with indices (NDWI by Gao and McFeeters, MNDWI by Xu)

## Getting the study area and parcels

In [ ]:
import geopandas as gpd
from sentineltimeseries.util.shapes import get_polygon_from_geojson

study_area = get_polygon_from_geojson('../resources/study_area/Polygon_WGS84.geojson')
parcels = gpd.read_file('../resources/study_area/AOI_BRP_WGS84.geojson')

## Getting the API

In [ ]:
from pyproj import CRS
from sentineltimeseries.api import SentinelTimeseriesAPI

api = SentinelTimeseriesAPI(
    username='your_copernicus_username',
    password='your_copernicus_password',
    aoi=study_area,
    warp=CRS('EPSG:4326'),
    working_directory='../resources/images'
)

## Adding the date for the Sentinel-2 product

In [ ]:
from datetime import date

sentinel12_products = api.get_sentinel2_products(date(2022, 1, 1), date(2022, 5, 1)) # Get a sentinel product from a specified date

In [ ]:
 parcels = parcels.assign(stat_count=0, stat_percentile95=0)

## Preprocessing the data and calculating the indices

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from rasterstats import zonal_stats
from sentineltimeseries.util.arrays import get_bands_as_arrays, mask_clouds_and_snow

for product in sentinel2_products:
    date_string = product.date.strftime("%Y-%m-%d")
    bands = product['B03', 'B08', 'CLD', 'SNW']
    band_arrays, affine_transform = get_bands_as_arrays(bands, api.aoi)
    band3, band8, cld, snw = band_arrays['B03'], band_arrays['B08'], band_arrays['CLD'], band_arrays['SNW']

    # Mask clouds and snow from band arrays
    band3_array = mask_clouds_and_snow(band3, cld, snw)
    band8_array = mask_clouds_and_snow(band8, cld, snw)

    # Calculate indice
    mndwi_xu = (band3_array - band8_array) / (band3_array + band8_array)

    # Calculate zonal statistics
    print(f"{date_string}: Calculating zonal statistics...")
    stats = zonal_stats(
        parcels,
        mndwi_xu.filled(),
        affine=affine_transform,
        nodata=-999,
        stats=['count', 'percentile_95']
    )

    # Calculate a running sum of statistics
    stat_count, stat_percentile95 = [], []
    for stat in stats:
        stat_count.append(stat['count'])
        # if there are no pixels we replace the statistics with 0
        # this should only be done, because we are summing statistics over multiple images
        stat_percentile95.append(stat['percentile_95'] if stat['percentile_95'] else 0)

    parcels['stat_count'] += stat_count
    parcels['stat_percentile95'] += stat_percentile95

## Average the zonal statistic over the amount of hotspots and plot the final image

In [ ]:
# Average the percentiles over the amount of hotspots
parcels['stat_percentile95'] = parcels['stat_percentile95'] / len(sentinel2_products)
# Plot average percentiles per parcel
fig, ax = plt.subplots(figsize=(12, 8))
parcels.plot(column='stat_percentile95', legend=True, cmap='Spectral', ax=ax)
plt.show()
